# Inspeccionar Modelos TBATS con StatsForecast

Este notebook muestra cómo acceder a los parámetros y resumen de modelos TBATS ajustados con `statsforecast`.

In [ ]:
# Instalación
!pip install statsforecast pandas numpy matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsforecast import StatsForecast
from statsforecast.models import AutoTBATS, TBATS
from google.colab import files

# Subir archivo
uploaded = files.upload()

In [ ]:
# Cargar y preparar datos
df = pd.read_csv('Champagne_Sales.csv', parse_dates=['Month'])
df.columns = ['ds', 'y']
df['unique_id'] = 'champagne'
df = df[['unique_id', 'ds', 'y']]

print("Datos cargados:")
print(df.head())

## Método 1: Inspeccionar el Modelo AutoTBATS

In [ ]:
# Ajustar modelo con AutoTBATS
model = StatsForecast(
    models=[AutoTBATS(season_length=12)],
    freq='MS',
    n_jobs=-1
)

print("🔄 Ajustando modelo AutoTBATS...\n")
model.fit(df)
print("✅ Modelo ajustado\n")

# Acceder al modelo ajustado
fitted_model = model.fitted_[0, 0]  # [serie, modelo]

print("=" * 70)
print("INFORMACIÓN DEL MODELO AJUSTADO")
print("=" * 70)

# 1. Tipo de modelo
print(f"\n📊 Tipo de modelo: {type(fitted_model).__name__}")

# 2. Atributos disponibles
print(f"\n📋 Atributos del modelo:")
attrs = [attr for attr in dir(fitted_model) if not attr.startswith('_')]
for attr in attrs[:20]:  # Mostrar primeros 20
    print(f"   - {attr}")

# 3. Intentar acceder a parámetros comunes
print(f"\n🔍 Parámetros identificados:")

# Nivel de serie
if hasattr(fitted_model, 'level'):
    print(f"   Level (nivel): {fitted_model.level}")

# Componente estacional
if hasattr(fitted_model, 'seasonal'):
    print(f"   Seasonal: {fitted_model.seasonal}")

# Parámetros del modelo
if hasattr(fitted_model, 'params'):
    print(f"   Params: {fitted_model.params}")

# Periodo estacional
if hasattr(fitted_model, 'season_length'):
    print(f"   Season length: {fitted_model.season_length}")

# AIC/BIC si están disponibles
if hasattr(fitted_model, 'aic'):
    print(f"   AIC: {fitted_model.aic:.2f}")
if hasattr(fitted_model, 'bic'):
    print(f"   BIC: {fitted_model.bic:.2f}")
if hasattr(fitted_model, 'aicc'):
    print(f"   AICc: {fitted_model.aicc:.2f}")

## Método 2: Inspeccionar con Model.__dict__

In [ ]:
print("\n" + "=" * 70)
print("TODOS LOS ATRIBUTOS INTERNOS DEL MODELO")
print("=" * 70 + "\n")

# Ver todos los atributos internos
for key, value in fitted_model.__dict__.items():
    print(f"📌 {key}:")
    if isinstance(value, (int, float, str, bool)):
        print(f"   {value}")
    elif isinstance(value, (list, tuple)) and len(value) < 10:
        print(f"   {value}")
    elif isinstance(value, np.ndarray):
        print(f"   Array de forma {value.shape}")
        if value.size < 20:
            print(f"   Valores: {value}")
    else:
        print(f"   Tipo: {type(value).__name__}")
    print()

## Método 3: Extraer Componentes del Modelo

In [ ]:
print("=" * 70)
print("COMPONENTES Y PREDICCIONES IN-SAMPLE")
print("=" * 70 + "\n")

# Hacer predicciones in-sample (fitted values)
fitted_values = model.forecast_fitted_values()

print("📊 Valores ajustados (fitted values):")
print(fitted_values.head(10))
print(f"\nForma: {fitted_values.shape}")
print(f"Columnas: {fitted_values.columns.tolist()}")

# Calcular residuos
residuals = df.set_index('ds')['y'] - fitted_values['AutoTBATS'].values

print(f"\n📈 Estadísticas de residuos:")
print(f"   Media: {residuals.mean():.2f}")
print(f"   Desviación estándar: {residuals.std():.2f}")
print(f"   Mínimo: {residuals.min():.2f}")
print(f"   Máximo: {residuals.max():.2f}")
print(f"   Mediana: {residuals.median():.2f}")

## Método 4: Visualizar Componentes del Modelo

In [ ]:
# Visualizaciones
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# 1. Datos originales vs ajustados
axes[0].plot(df['ds'], df['y'], label='Datos Originales', linewidth=2, color='blue', alpha=0.7)
axes[0].plot(fitted_values.index, fitted_values['AutoTBATS'], 
            label='Valores Ajustados', linewidth=1.5, color='red', linestyle='--', alpha=0.8)
axes[0].set_title('Datos Originales vs Valores Ajustados', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Fecha')
axes[0].set_ylabel('Ventas')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Residuos
axes[1].plot(df['ds'], residuals, linewidth=1, color='purple', alpha=0.6)
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1].fill_between(df['ds'], residuals, 0, alpha=0.3, color='purple')
axes[1].set_title('Residuos del Modelo', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Fecha')
axes[1].set_ylabel('Residuo')
axes[1].grid(True, alpha=0.3)

# 3. Distribución de residuos
axes[2].hist(residuals, bins=20, color='purple', alpha=0.6, edgecolor='black')
axes[2].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Media=0')
axes[2].set_title('Distribución de Residuos', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Residuo')
axes[2].set_ylabel('Frecuencia')
axes[2].legend()
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Método 5: Usar la Librería TBATS Original (si necesitas más detalles)

In [ ]:
# Si realmente necesitas el resumen completo, usa tbats original
print("\n" + "=" * 70)
print("COMPARACIÓN: USANDO TBATS ORIGINAL PARA RESUMEN COMPLETO")
print("=" * 70 + "\n")

# Primero verifica si está instalado
try:
    from tbats import TBATS as TBATS_Original
    
    # Convertir datos
    y_values = df['y'].values
    
    # Ajustar modelo
    print("🔄 Ajustando TBATS original para comparación...\n")
    estimator = TBATS_Original(
        seasonal_periods=[12],
        use_box_cox=True,
        use_trend=True,
        use_damped_trend=False
    )
    fitted_original = estimator.fit(y_values)
    
    print("✅ TBATS original ajustado\n")
    print("📋 RESUMEN COMPLETO DEL MODELO TBATS ORIGINAL:")
    print("=" * 70)
    print(fitted_original.summary())
    print("=" * 70)
    
    # Extraer parámetros específicos
    print("\n🔍 Parámetros detallados:")
    print(f"   Box-Cox lambda: {fitted_original.lambda_}")
    print(f"   Componentes ARMA: p={fitted_original.p}, q={fitted_original.q}")
    print(f"   Períodos estacionales: {fitted_original.seasonal_periods}")
    print(f"   Usa tendencia: {fitted_original.use_trend}")
    print(f"   Usa Box-Cox: {fitted_original.use_box_cox}")
    print(f"   AIC: {fitted_original.aic:.2f}")
    
except ImportError:
    print("⚠️ Para ver el resumen completo, instala tbats original:")
    print("   1. Runtime → Restart runtime")
    print("   2. !pip install 'numpy<2.0' tbats")
    print("   3. Ejecuta esta celda de nuevo")
except Exception as e:
    print(f"❌ Error al usar tbats original: {e}")

## Método 6: Crear Resumen Personalizado con StatsForecast

In [ ]:
# Crear un resumen personalizado basado en métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("\n" + "=" * 70)
print("RESUMEN PERSONALIZADO DEL MODELO STATSFORECAST")
print("=" * 70 + "\n")

# Obtener valores ajustados
y_true = df['y'].values
y_fitted = fitted_values['AutoTBATS'].values
residuals_array = y_true - y_fitted

# Calcular métricas
mse = mean_squared_error(y_true, y_fitted)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_fitted)
mape = np.mean(np.abs(residuals_array / y_true)) * 100
r2 = r2_score(y_true, y_fitted)

# Estadísticas de residuos
ljung_box_stat = "N/A (calcular manualmente si se necesita)"

print("📊 MÉTRICAS DE AJUSTE")
print("-" * 70)
print(f"MSE (Error Cuadrático Medio):        {mse:>15,.2f}")
print(f"RMSE (Raíz del MSE):                 {rmse:>15,.2f}")
print(f"MAE (Error Absoluto Medio):          {mae:>15,.2f}")
print(f"MAPE (Error Porcentual Absoluto):    {mape:>15,.2f}%")
print(f"R² (Coeficiente de Determinación):  {r2:>15,.4f}")

print("\n📈 ESTADÍSTICAS DE RESIDUOS")
print("-" * 70)
print(f"Media:                               {residuals_array.mean():>15,.2f}")
print(f"Desviación Estándar:                 {residuals_array.std():>15,.2f}")
print(f"Varianza:                            {residuals_array.var():>15,.2f}")
print(f"Mínimo:                              {residuals_array.min():>15,.2f}")
print(f"Máximo:                              {residuals_array.max():>15,.2f}")
print(f"Percentil 25:                        {np.percentile(residuals_array, 25):>15,.2f}")
print(f"Mediana (Percentil 50):              {np.median(residuals_array):>15,.2f}")
print(f"Percentil 75:                        {np.percentile(residuals_array, 75):>15,.2f}")

print("\n⚙️ CONFIGURACIÓN DEL MODELO")
print("-" * 70)
print(f"Tipo:                                {'AutoTBATS':>15}")
print(f"Periodo Estacional:                  {12:>15}")
print(f"Frecuencia:                          {'Mensual (MS)':>15}")
print(f"Número de Observaciones:             {len(df):>15}")

print("\n✅ INTERPRETACIÓN")
print("-" * 70)
if mape < 10:
    print("✓ Excelente ajuste (MAPE < 10%)")
elif mape < 20:
    print("✓ Buen ajuste (MAPE < 20%)")
elif mape < 50:
    print("⚠ Ajuste aceptable (MAPE < 50%)")
else:
    print("⚠ Considerar mejorar el modelo (MAPE > 50%)")

if abs(residuals_array.mean()) < residuals_array.std() * 0.1:
    print("✓ Residuos bien centrados (media cercana a 0)")
else:
    print("⚠ Posible sesgo en residuos")

if r2 > 0.9:
    print(f"✓ Excelente capacidad explicativa (R² = {r2:.4f})")
elif r2 > 0.7:
    print(f"✓ Buena capacidad explicativa (R² = {r2:.4f})")
else:
    print(f"⚠ Capacidad explicativa moderada (R² = {r2:.4f})")

print("\n" + "=" * 70)

## Conclusión

### Limitaciones de StatsForecast:
- ❌ No proporciona un `summary()` detallado como tbats original
- ❌ Acceso limitado a parámetros internos del modelo
- ❌ No expone directamente componentes (tendencia, estacionalidad)

### Ventajas:
- ✅ Más rápido
- ✅ Sin problemas de compatibilidad
- ✅ Puedes calcular todas las métricas importantes manualmente

### Recomendación:
- Para **análisis exploratorio**: Usa StatsForecast
- Para **análisis detallado de componentes**: Usa tbats original (con las correcciones de NumPy)
- Para **producción**: Usa StatsForecast y calcula métricas personalizadas